# Industry Accelerators - Customer Life Event Prediction Models

## Introduction

Now that we have built the machine learning models, stored and deployed them, we can use the models to score new data. 



Before executing this notebook on IBM Cloud, you need to:<br>
1) When you import this project on an IBM Cloud environment, a project access token should be inserted at the top of this notebook as a code cell. <br>
If you do not see the cell above, Insert a project token: Click on **More -> Insert project token** in the top-right menu section and run the cell. <br>

![ws-project.mov](https://media.giphy.com/media/jSVxX2spqwWF9unYrs/giphy.gif)
2) Provide your IBM Cloud API key in the subsequent cell.<br>
3) You can then step through the notebook execution cell by cell, by selecting Shift-Enter. Or you can execute the entire notebook by selecting **Cell -> Run All** from the menu.<br>


#### Insert IBM Cloud API key
Your Cloud API key can be generated by going to the [API Keys section of the Cloud console](https://cloud.ibm.com/iam/apikeys). From that page, scroll down to the API Keys section, and click Create an IBM Cloud API key. Give your key a name and click Create, then copy the created key and paste it below. 

If you are running this notebook on cloud pak for data on-prem, leave the ibmcloud_api_key field blank.

In [2]:
ibmcloud_api_key = ''

In [3]:

try:
    project
except NameError:
    # READING AND WRITING PROJECT ASSETS
    import project_lib
    project = project_lib.Project() 

# Create and Test Scoring Pipeline
In the notebook we will:

* Programmatically get the ID's for the deployment space and model deployments that were created in the **2-model-training** notebook
* Promote assets required for scoring new data into the deployment space
* Create a deployable function which will take raw data for scoring, prep it into the format required for the models and score it
* Deploy the function
* Create the required payload, invoke the deployed function and return predictions

### User Inputs

Enter the user's wml credentials, the path to the csv file with raw data to be scored and a list of events to be predicted for.

In [4]:

import os
import pandas as pd
import datetime
import json
from ibm_watson_machine_learning import APIClient
import os

if ibmcloud_api_key != '':
    wml_credentials = {
        "apikey": ibmcloud_api_key,
        "url": 'https://' + os.environ['RUNTIME_ENV_REGION'] + '.ml.cloud.ibm.com'
    }
else:
    token = os.environ['USER_ACCESS_TOKEN']
    wml_credentials = {
        "token": token,
        "instance_id" : "openshift",
        "url": os.environ['RUNTIME_ENV_APSX_URL'],
        "version": "3.5"
     }
client = APIClient(wml_credentials)

### User Inputs

Enter the prediction events to be scored.

In [5]:
prediction_events = ['LFE_RELOCATION', 'LFE_HOME_PURCHASE']

### Set up Deployment Space, Deployments and Assets

The following code programmatically gets the deployment space and the model deployment details which were created in **2-model-training**. 
We use the space name and deployment names that were used when creating the deployments, as specified below. 
If multiple deployments within the selected space have the same name, the most recently created deployment is used. 

Alternatively, the user can manually enter the space and deployment id's.

The code also promotes some assets into the deployment space, specifically, the dataset with raw data for scoring, the python script file which is used for prepping the data and the metadata that was stored when prepping the data. By promoting these assets into the deployment space, they are available and can be accessed by the deployed function. 

In [6]:
space_name = 'Customer Life Event Space'

model_names = {}
deployment_names = {}
for event_type in prediction_events: 
    model_names[event_type] = 'life_event_' + event_type + '_model'
    deployment_names[event_type] = 'life_event_' + event_type + '_model_deployment'

Get the space we are working in, which is found using the name that were hardcoded in **2-model_training**. 

If you would like to use a different space, manually set the space_id.

Set the space as the default space for working.

In [7]:
l_space_details = []
l_space_details_created_times = []
for space_details in client.spaces.get_details()['resources']:
    if space_details['entity']['name'] == space_name:
        space_id=space_details['metadata']['id']

# set this space as default space
client.set.default_space(space_id)

'SUCCESS'

In [8]:
for event_type, _ in deployment_names.items():
    print(event_type)

LFE_RELOCATION
LFE_HOME_PURCHASE


Get the deployment id. If there are multiple deployments with the same name in the same space, we take the latest.

In [9]:
model_deployments_dict = {}
for event_type, deployment_name in deployment_names.items():
    # get the id of the deployments - again, found using the tags that were hardcoded
    # if there are multiple deployments with the same tag in the same space, we take the latest
    l_deployment_details = []
    l_deployment_details_created_times = []
    for deployment in client.deployments.get_details()['resources']:
            if deployment['entity']['name'] == deployment_names[event_type]:            
                l_deployment_details.append(deployment)
                l_deployment_details_created_times.append(datetime.datetime.strptime(deployment['metadata']['created_at'],  '%Y-%m-%dT%H:%M:%S.%fZ'))

    # get the index of the latest created date from the list and use that to get the deployment_id
    list_latest_index = l_deployment_details_created_times.index(max(l_deployment_details_created_times))
    deployment_id = l_deployment_details[list_latest_index]['metadata']['id']
    model_deployments_dict[event_type] = deployment_id
print("Deployments :",model_deployments_dict)

Deployments : {'LFE_RELOCATION': 'f1467555-3131-4efa-a084-b88bf5afa9bb', 'LFE_HOME_PURCHASE': 'e3fd5328-ef37-4ba4-bd1d-d111b3e3ac6e'}



We will use the prep script for getting the raw data into the format required for scoring.

Additionally, we also need the prep metadata that was saved as json during the prep for training - this ensures that the user inputs specified for prepping the data for training are the same that are used for scoring.

Finally, we need to add these files into the deployment space.

### Promote Assets to Deployment space

Promote the assets into the deployment space. We will use the prep script for getting the raw data into the format required for scoring. We also need the prep metadata that was saved as json during the prep for training. This ensures that the user inputs specified for prepping the data for training are the same as the ones used for scoring. We add these assets into the deployment space.  We also store the raw data dataset in the deployment space.

In [10]:
asset_details_json = client.data_assets.create('training_user_inputs_and_prepped_column_names.json', file_path='training_user_inputs_and_prepped_column_names.json')
asset_details_script = client.data_assets.create('life_event_prep.py', file_path='life_event_prep.py')
# add the census prep script
asset_details_census_script = client.data_assets.create('prep_census_data.py', file_path='prep_census_data.py')
# also store the dataset in the deployment space
asset_details_dataset = client.data_assets.create('event.csv', file_path='event.csv')
# add the census dataset the deployment space
asset_details_census_dataset = client.data_assets.create('census_probabilities.csv', file_path='census_probabilities.csv')
# the census prep also requires the customer csv file
asset_details_customer_dataset = client.data_assets.create('customer.csv', file_path='customer.csv')

Creating data asset...
SUCCESS
Creating data asset...
SUCCESS
Creating data asset...
SUCCESS
Creating data asset...
SUCCESS
Creating data asset...
SUCCESS
Creating data asset...
SUCCESS


In [11]:
client.data_assets.list()

--------------------------------------------------  ----------  ---------  ------------------------------------
NAME                                                ASSET_TYPE  SIZE       ASSET_ID
customer.csv                                        data_asset  1273147    c63ed47a-d41e-43e0-b347-310e8782381e
life_event_prep.py                                  data_asset  37350      dc6d6c5a-0a2e-4ac5-834c-fc13eee26236
customer.csv                                        data_asset  1273147    8e4640e6-e513-44ae-819c-fcc40d568b5c
life_event_prep.py                                  data_asset  37350      8b8b9a3c-0476-400f-bb71-441114c5f442
event.csv                                           data_asset  3088316    0b8cdfcf-3471-4aff-b29d-71441b1998fc
training_user_inputs_and_prepped_column_names.json  data_asset  2747       e5195fde-52d6-45fa-9dbd-6cf795379f0f
prep_census_data.py                                 data_asset  8433       fdb6bcfd-6cf4-4d7c-b497-80207fb86c61
event.csv           

## Create the Deployable Function

Functions can be deployed in Watson Machine Learning in the same way that models can be deployed. The python client or REST API can be used to send data to the deployed function. Using the deployed function allows us to prepare the data and pass it to the model for scoring; all within the deployed function.

We start off by creating the dictionary of default parameters to be passed to the function. We get the ID's of all assets that have been promoted into the deployment space. We also add the model deployment ID and space ID  information into the dictionary.

Get the guid's of the assets that were stored in the space. Include these in the dictionary of default parameters that are passed to the function.

In [12]:
# get the assets that were stored in the space - in this version of the package we need to manually assign the id

metadata_id = asset_details_json['metadata']['guid']
prep_id = asset_details_script['metadata']['guid']
dataset_id = asset_details_dataset['metadata']['guid']
census_dataset_id = asset_details_census_dataset['metadata']['guid']
census_prep_id = asset_details_census_script['metadata']['guid']
customer_dataset_id = asset_details_customer_dataset['metadata']['guid']

In [13]:
assets_dict = {'dataset_asset_id' : dataset_id, 'metadata_asset_id' : metadata_id, 'census_asset_id' : census_dataset_id, 'customer_asset_id' : customer_dataset_id,
                   'census_prep_script_asset_id' : census_prep_id, 'prep_script_asset_id' : prep_id, 'dataset_name' : 'event.csv', 
                   'census_dataset_name' : 'census_probabilities.csv', 'customer_dataset_name' : 'customer.csv'}

In [14]:
ai_parms = {'wml_credentials' : wml_credentials, 'space_id' : space_id,  'assets' : assets_dict, 'model_deployment_id' : model_deployments_dict}

### Scoring Pipeline Function

The function below takes new customers to be scored as payload. It preps the customer raw data, loads the model, executes model scoring and generates predictions for life events. 


The following rules are required to make a valid deployable function:

> * The deployable function must include a nested function named `"score"`.
> * The score function accepts a list.
> * The list must include an array with the name `"values"`.
> * The score function must return an array with the name `"predictions"`, with a list as the value, which in turn contains an array with the name `"values". Example: {"predictions" : [{'values' : }]}`
> * We pass the following into the function: default parameters, credentials and space details, details of the assets that were promoted into the space, and the model deployment guid.
> * Assets are downloaded into the deployment space and imported as variables. The raw data to be scored is then prepared and the function calls the model deployment endpoints to score and return predictions.

In [15]:
def scoring_pipeline(parms=ai_parms):
     
    import pandas as pd
    import requests
    import os
    import json
    
    from ibm_watson_machine_learning import APIClient
    client = APIClient(parms["wml_credentials"])
    client.set.default_space(parms['space_id'])

    # call the function to download the stored dataset asset and return the path
    dataset_path = client.data_assets.download(parms['assets']['dataset_asset_id'], parms['assets']['dataset_name'])
    df_raw = pd.read_csv(dataset_path, infer_datetime_format=True, 
                             parse_dates=['EVENT_DATE'])
    
    # download the census data 
    census_dataset_path = client.data_assets.download(parms['assets']['census_asset_id'], parms['assets']['census_dataset_name'])
    df_census = pd.read_csv(census_dataset_path)
    # download the customer dataset
    customer_dataset_path = client.data_assets.download(parms['assets']['customer_asset_id'], parms['assets']['customer_dataset_name'])
    df_customer = pd.read_csv(customer_dataset_path)
    
    # call the function to download the prep script and return the path
    prep_script_path = client.data_assets.download(parms['assets']['prep_script_asset_id'], 'prep_data_script.py')
    # remove the rest of path and .py at end of file name to get the name of the script for importing
    script_name = os.path.basename(prep_script_path).replace('.py', '')
    # call the function to downlaod the census prep script and return the path
    census_prep_script_path = client.data_assets.download(parms['assets']['census_prep_script_asset_id'], 'census_prep_data_script.py')
    # remove the rest of path and .py at end of file name to get the name of the script for importing
    census_script_name = os.path.basename(census_prep_script_path).replace('.py', '')
    
    # call the function to download the prep metadata and return the path
    metadata_path = client.data_assets.download(parms['assets']['metadata_asset_id'], 'user_inputs.json')
    
    def prep(cust_id, sc_end_date):
        import requests
        import os
        # import the prep script that we downloaded into the deployment space
        prep_data_script = __import__(script_name)
    
        with open(metadata_path, 'r') as f:
            user_inputs_dict = json.load(f)
        
        globals().update(user_inputs_dict)
        
        input_df = df_raw[df_raw['CUSTOMER_ID'] == cust_id]
        
        scoring_prep = prep_data_script.LifeEventPrep(target_event_type_ids, train_or_score='score',
                                 b_use_census_data=b_use_census_data,
                                 scoring_end_date=sc_end_date, forecast_horizon=forecast_horizon,
                                 observation_window=observation_window,
                                 life_event_minimum_target_count=life_event_minimum_target_count,
                                 norepeat_months=norepeat_months,cols_to_drop=cols_to_drop)
        
        prepped_data_dict = scoring_prep.prep_data(input_df, 'score')
        
        for event_type in target_event_type_ids:
            if prepped_data_dict[event_type] is None:
                print("Data prep filtered out customer data. Unable to score.", file=sys.stderr)
                return None

            # handle empty data
            if prepped_data_dict[event_type].shape[0] == 0:
                print("Data prep filtered out customer data. Unable to score.", file=sys.stderr)
                return None
        
            # don't need to include target variable for scoring
            cols_used_for_training[event_type].remove('TARGET')
        
            # if a column does not exist in scoring but is in training, add the column to scoring dataset
            for col in cols_used_for_training[event_type]:
                if col not in list(prepped_data_dict[event_type].columns):
                    prepped_data_dict[event_type][col] = 0

            # if a column exists in scoring but not in training, delete it from scoring dataset
            for col in list(prepped_data_dict[event_type].columns):
                if col not in cols_used_for_training[event_type]:
                    prepped_data_dict[event_type].drop(col, axis=1, inplace=True)

            # make sure order of scoring columns is same as training dataset
            prepped_data_dict[event_type] = prepped_data_dict[event_type][cols_used_for_training[event_type]]
        
        if b_use_census_data:
            census_script = __import__(census_script_name)
            census = census_script.census_data()
            prepped_data_dict = census.prep_census(df_census,df_customer,prepped_data_dict,'score')
    
        return prepped_data_dict
    
    def score(payload):
        import json
        
        sc_end_date = payload['input_data'][0]['values']
        cust_id = payload['input_data'][0]['cust_id']
        
        prepped_data_dict = prep(cust_id, sc_end_date)
        
        result = {}
        
        
        for event_type, prepped_data in prepped_data_dict.items():
            # handle empty data
            if prepped_data is None:
                return {"predictions" : [{'values' : 'Data prep filtered out customer data. Unable to score.'}]}
            elif prepped_data.shape[0] == 0:
                return {"predictions" : [{'values' : 'Data prep filtered out customer data. Unable to score.'}]}
            else:
                
                scoring_payload = {"input_data":  [{ "values" : prepped_data.values.tolist()}]}
                
                response_scoring = client.deployments.score(parms['model_deployment_id'][event_type], scoring_payload)
                result[event_type] = response_scoring  
        
        return {"predictions" : [{'values' : result}]}
    
    return score

### Deploy the Function

The user can specify the name of the function and deployment in the code below. As we have previously seen, we use tags in the metadata to allow us to programmatically identify the deployed function. 

In [16]:
# store the function and deploy it 
function_name = 'life_event_scoring_pipeline_function'
function_deployment_name = 'life_event_scoring_pipeline_function_deployment'

### Get the ID of software specification to be used with the function

The Software Specification refers to the runtime used in the Notebook, WML training and WML deployment. It contains details about the runtime platform, framework versions, other packages used and any custom library used in the concerned runtime.

When we deploy our function we want it to have the same software specification as the notebooks. 

We use the `default_py3.7` software specification to store the function. We get the ID of the software specification and include it in the metadata when storing the function. Available Software specifications can be retrieved using `client.software_specifications.list()`.


In [17]:
software_spec_id = client.software_specifications.get_id_by_name("default_py3.7")

In [18]:
# add the metadata for the function and deployment    
meta_data = {
    client.repository.FunctionMetaNames.NAME : function_name,
    client.repository.FunctionMetaNames.TAGS : ['lfe_scoring_pipeline_function_tag'],
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: software_spec_id
}

function_details = client.repository.store_function(meta_props=meta_data, function=scoring_pipeline)

function_id = function_details["metadata"]["id"]

meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: function_deployment_name,
    client.deployments.ConfigurationMetaNames.TAGS : ['lfe_scoring_pipeline_function_deployment_tag'],
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

# deploy the stored model
function_deployment_details = client.deployments.create(artifact_uid=function_id, meta_props=meta_props)



#######################################################################################

Synchronous deployment creation for uid: '4c3b0f3c-76d0-47b7-bf6d-624b381bae8e' started

#######################################################################################


initializing.......
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='4172ac90-f899-4ca6-a1c2-d32137aef39b'
------------------------------------------------------------------------------------------------




### Score New Data

Get the guid of the deployed function, create the payload and use the python client to score the data. The deployed function returns the classification prediction along with the probabilities. 

The payload contains two values. The first is the effective date for scoring. This is the date that the prediction is computed. The scoring observation window and forecast horizon are calculated from this date. The second value contains the ID of the customer who we would like to make the prediction for. 

In [19]:
scoring_deployment_id = client.deployments.get_uid(function_deployment_details)
client.deployments.get_details(scoring_deployment_id)

{'entity': {'asset': {'id': '4c3b0f3c-76d0-47b7-bf6d-624b381bae8e'},
  'custom': {},
  'deployed_asset_type': 'function',
  'hardware_spec': {'id': 'Not_Applicable', 'name': 'XS', 'num_nodes': 1},
  'name': 'life_event_scoring_pipeline_function_deployment',
  'online': {},
  'space_id': '6e013c84-ac66-4a53-b503-d049e90e9821',
  'status': {'online_url': {'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/4172ac90-f899-4ca6-a1c2-d32137aef39b/predictions'},
   'state': 'ready'}},
 'metadata': {'created_at': '2021-06-04T14:34:44.335Z',
  'id': '4172ac90-f899-4ca6-a1c2-d32137aef39b',
  'modified_at': '2021-06-04T14:34:44.335Z',
  'name': 'life_event_scoring_pipeline_function_deployment',
  'owner': 'IBMid-550003B08R',
  'space_id': '6e013c84-ac66-4a53-b503-d049e90e9821',
  'tags': ['lfe_scoring_pipeline_function_deployment_tag']}}

In [20]:
cust_id = 1007
payload = [{'values' : "2018-09-30", 'cust_id' : cust_id}]

payload_metadata = {client.deployments.ScoringMetaNames.INPUT_DATA: payload}
# score
funct_output = client.deployments.score(scoring_deployment_id, payload_metadata)
funct_output

{'predictions': [{'values': {'LFE_RELOCATION': {'predictions': [{'fields': ['prediction',
        'probability'],
       'values': [[0, [0.7932749402712382, 0.2067250597287618]]]}]},
    'LFE_HOME_PURCHASE': {'predictions': [{'fields': ['prediction',
        'probability'],
       'values': [[0, [0.9683913063113182, 0.03160869368868198]]]}]}}}]}

**The R Shiny Dashboard invokes this scoring pipeline for visualizing the results.**<br>
**Follow the instructions from Readme to launch the R-Shiny dashboard.**


<hr>

Sample Materials, provided under <a href="https://github.com/IBM/Industry-Accelerators/blob/master/CPD%20SaaS/LICENSE" target="_blank" rel="noopener noreferrer">license.</a> <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2019, 2021. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp. <br>